# Temporary sandbox notebook
Only for integrating DOS plot into bandplot experiments.

Done here so as not to mess up the main notebook.

When done and stable, integrate changes back into main development notebook.

# Setup

## Setup masci-tools path

In [1]:
# IMPORTANT: we need to import stuff from masci-tools folder.
# Since masci-tools is not installed as a module (yet), the notebook kernel
# needs to be started in the masci-tools folder.
# If that has not happened for some reason, then need to add the masci-tools
# manually to the sys path.
import os
import sys

cwd = os.getcwd()
path_mtools = cwd
dirname_mtools = "masci-tools"
# first try if we can get away without needing an absolute path
if dirname_mtools in path_mtools:
    while os.path.basename(path_mtools) != dirname_mtools:
        path_mtools = os.path.split(path_mtools)[0]
else:
    # okay, try with an absolute path
    path_mtools = "/home/johannes/Desktop/Studium/Kurse_RWTH/SiScLab/18W/repos/masci-tools"
    if not os.path.isdir(path_mtools):
        raise IOError(f"Could not find path to masci-tools. Please specify absolute path.")

# found masci-tools. add to syspath (for imports) and chdir.
if path_mtools not in sys.path:
    # add only once
    sys.path.append(path_mtools)

## Setup imports

In [2]:
# Jupyter, Python imports
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

#  python 3interactive figures in a live IPython notebook session
# if run from jupyter-notebook: MAGICmatplotlib nbagg
# if run from jupyter-lab: MAGICmatplotlib widget
%matplotlib widget

# studentproject18ws imports
import os
import logging
from studenproject18ws.hdf.reader import Reader
from studenproject18ws.hdf.recipes import Recipes
from studenproject18ws.plot.plot import Bandplot_matplotlib as Bandplot

In [3]:
%matplotlib widget

## Read file, import data

- TODO: link with widget-based file-chooser

In [4]:
# # NO DOS file:
# filename = 'banddos.hdf'
# filename = 'banddos_4x4.hdf'
# filename = 'banddos_sodium.hdf'

# 1 DOS file:
filename = os.path.join('MoSe2', 'banddos_2spin.hdf')
filenames_dos = [os.path.join('MoSe2',"DOS.1")]

# # 2 DOS files:
# filename = os.path.join('Co', 'banddos_Co.hdf')
# filenames_dos = [os.path.join('Co', 'DOS.1'), os.path.join('Co', 'DOS.2')]

filepath = ['..', 'data', 'input', filename]
filepath = os.path.join(*filepath)
filepaths_dos = [['..', 'data', 'input', fd] for fd in filenames_dos]
filepaths_dos = [os.path.join(*fpd) for fpd in filepaths_dos]

data = None
reader = Reader(filepath=filepath)
with reader as h5file:
    data = reader.read(recipe=Recipes.Bands)
    #
    # Note:
    # Inside the with statement (context manager),
    # all data attributes that are type h5py Dataset are available (in-file access)
    # When the statement is left,the HDF5 file gets closed and the datasets are closed.
    #
    # Use data outside the with-statement (in-memory access: all HDF5 datasets converted to numpy ndarrays):
    data.move_datasets_to_memory()

### Plotter class

In [5]:
bandplotter = Bandplot(data)
        

# Testing: compare DOS plotting methods without band plot

In [6]:
def plot_dos_old(ax_dos, filepath_dos):
    dos_data = np.genfromtxt(filepath_dos).T
    energy_dos = dos_data[0]
    totdos = dos_data[1]
    interst = dos_data[2]
    vac1 = dos_data[3]
    vac2 = dos_data[4]
    weights_atomgrps_dos = dos_data[5:]

#     ax_dos = fig.add_subplot(111)
    ax_dos.plot(totdos, energy_dos)
    ax_dos.plot(interst, energy_dos)

    #constant 0
    ax_dos.plot(vac1, energy_dos)
    ax_dos.plot(vac2, energy_dos)

    ax_dos.plot((sum(weights_atomgrps_dos)+interst), energy_dos)
#     plt.show()


# compare our plot_dos (OLD one as of 17.01.19), and masci-tool's plot_dos output.
fig, ax = plt.subplots(1)
plot_dos_old(ax, filepaths_dos[0])
plt.show()

plt.clf()
fig, ax = plt.subplots(1)
sel = data.simulate_gui_selection()
bandplotter.plot_dos(filepaths_dos[0], sel.mask_groups, sel.mask_characters,
           select_groups=True, interstitial=True, all_characters=True,
               ax=ax)
# bandplotter.plot_dos(filepaths_dos[0])
plt.show()


from masci_tools.vis.plot_methods import plot_dos as plot_dos_masci
plot_dos_masci(filepaths_dos[0])



FigureCanvasNbAgg()

FigureCanvasNbAgg()

get_dos: for selection groups [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True], characters [False False False False], num_groups 18, atoms_per_group [2. 2. 2. 1. 2. 2. 2. 1. 1. 2. 1. 2. 2. 1. 1. 1. 1. 1.], accessed dos columns: [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


FigureCanvasNbAgg()

# Define Widgets

## Band plot

### Define user input arguments

In [6]:
# bands = atom_group_keys = e.g. for banddos.hdf: dict_keys[(1,2,3,4,5)]
#                           Hm... should better convert to tuple back in reader?
def_groups = data.atoms_group_keys
select_groups = widgets.SelectMultiple(options=def_groups, 
                                   value=tuple(def_groups),
                                  description='Atom Groups',
                                  disabled=False)


def_characters = ['s', 'p', 'd', 'f']
# Characters = namedtuple('Characters', ['s', 'p', 'd', 'f'])
# characters = Characters(0,1,2,3)
select_characters = widgets.SelectMultiple(options=def_characters,
                                          value=tuple(def_characters),
                                          description='Band Character',
                                          disabled=False)

# number of bands can be large, so use a rangeslider instead of selectionslider
def_bands = [band for band in range(data.eigenvalues.shape[2])]
select_bands = widgets.IntRangeSlider(value=[def_bands[0]+1,def_bands[-1]+1], 
                                      min=def_bands[0]+1, max=def_bands[-1]+1, step=1,
                                     description='Bands',
                                     disabled=False, continuous_update=False,
                                     orientation='horizontal', readout=True,
                                     readout_format='d')

select_exponent = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=4.0,
    step=0.01,
    description='Unfolding weight exponent',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f')

select_marker_size = widgets.FloatSlider(
    value=1.0,
    min=0,
    max=10.0,
    step=0.01,
    description='Marker size',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f')

select_compare_characters = widgets.Checkbox(
    value=False,
    description="Compare 2 characters (disabled)",
    disabled=True
)

# make compare_characters only available when 2 characters are selected
def on_character_selection_change(change):
    if len(change.new) == 2:
        select_compare_characters.disabled = False
        select_compare_characters.description = "Compare 2 characters (enabled)"
    else:
        select_compare_characters.disabled = True
        select_compare_characters.description = "Compare 2 characters (disabled)"
select_characters.observe(on_character_selection_change, names='value')

# when compare_characters is active, disable select_characters
def on_compare_character(change):
    select_characters.disabled = change.new
select_compare_characters.observe(on_compare_character, names='value')

def convert_selections(bands=[], characters=[], groups=[]):
    # convert arguments to the expected format for code 181124
    bands_conved = range(bands[0]-1,bands[1]) if bands else []
    groups_conved = [el-1 for el in groups] if groups else []
    characters_conved = [def_characters.index(el) for el in characters] if characters else []
    
    # convert arguments to the expected format for code 181212
    mask_characters = [el in characters for el in def_characters] if characters else []
    mask_bands = [el in bands_conved for el in def_bands] if bands else []
    mask_groups = [el in [el for el in groups] for el in def_groups] if groups else []
    
    return (mask_bands, mask_characters, mask_groups)

## Try out the new DOS plot method interactively without band plot

In [8]:
spin = 0

fig_scale = 0.65
fig_ratio = [10,6]
fig, ax = plt.subplots(1, figsize=[fig_scale * el for el in fig_ratio])
plt.suptitle(f"DOS of {filenames_dos[spin]}")

# plt.clf()
# fig, ax = plt.subplots(1)
# sel = data.simulate_gui_selection()
# bandplotter.dos(filepaths_dos[0], sel.mask_groups, sel.mask_characters,
#            select_groups=True, interstitial=True, all_characters=True,
#                ax=ax)
# plt.show()

@interact(characters=select_characters, groups=select_groups, dos_groups_yesno=True, interstitial=True, all_characters=True)
def update_plot_dos_test(characters, groups, dos_groups_yesno, interstitial, all_characters):
    ax.clear()
    (mask_bands, mask_characters, mask_groups) = convert_selections(bands=[], characters=characters, groups=groups)
    bandplotter.plot_dos(filepaths_dos[0], mask_groups, mask_characters,
           dos_groups_yesno, interstitial, all_characters,
               ax=ax, fix_xlim=True)


FigureCanvasNbAgg()

interactive(children=(SelectMultiple(description='Band Character', index=(0, 1, 2, 3), options=('s', 'p', 'd',…

## Interact function

In [9]:
fig_scale = 0.65
fig_ratio = [10,6]
fig, (ax, ax_dos) = plt.subplots(1, 2, sharey=True, 
                                 figsize=[fig_scale * el for el in fig_ratio])
ax_dos.set_aspect('auto') #default: 'auto'. other opts: 'equal', ratio (height, number)


plt.suptitle(f"BandStructure of {filename}")
fig.subplots_adjust(wspace=0)

plot_dos_old(ax_dos, filepaths_dos[0])


@interact(bands=select_bands, characters=select_characters, groups=select_groups, 
          unfolding_weight_exponent=select_exponent, marker_size=select_marker_size,
         compare_characters=select_compare_characters,
         dos_groups_yesno=True, interstitial=True, all_characters=True)
def update_plot(bands, characters, groups, 
                unfolding_weight_exponent, marker_size,
               compare_characters,
               dos_groups_yesno, interstitial, all_characters):
    ax.clear()
    ax_dos.clear()
#     bandplotter.setup(ax)

    (mask_bands, mask_characters, mask_groups) = convert_selections(bands, characters, groups)

    # TODO: switch button for ignore_atoms_per_group
    ignore_atoms_per_group = True #default
    spins = [0]
    
    bandplotter.plot_bands(mask_bands, mask_characters, mask_groups, spins, 
         unfolding_weight_exponent, compare_characters, ax, ignore_atoms_per_group, marker_size)
    bandplotter.plot_dos(filepaths_dos[0], mask_groups, mask_characters,
           dos_groups_yesno, interstitial, all_characters,
               ax=ax_dos, fix_xlim=True)

FigureCanvasNbAgg()

interactive(children=(IntRangeSlider(value=(1, 303), continuous_update=False, description='Bands', max=303, mi…

In [10]:
# Trying new layout with gridspec.
# Reference: https://plot.ly/matplotlib/subplots/


from matplotlib import gridspec

fig_scale = 0.65
fig_ratio = [10,6]
figsize=[fig_scale * el for el in fig_ratio]

fig = plt.figure()
gs = gridspec.GridSpec(1, 2)

ax = fig.add_subplot(gs[0,0])
ax_dos = fig.add_subplot(gs[0,1])
gs.update(wspace=0, hspace=0)


# fig, (ax, ax_dos) = plt.subplots(1, 2, sharey=True, 
#                                  figsize=[fig_scale * el for el in fig_ratio])



plt.suptitle(f"BandStructure of {filename}")


@interact(bands=select_bands, characters=select_characters, groups=select_groups, 
          unfolding_weight_exponent=select_exponent, marker_size=select_marker_size,
         compare_characters=select_compare_characters,
         dos_groups_yesno=True, interstitial=True, all_characters=True)
def update_plot(bands, characters, groups, 
                unfolding_weight_exponent, marker_size,
               compare_characters,
               dos_groups_yesno, interstitial, all_characters):
    ax.clear()
    ax_dos.clear()
#     bandplotter.setup(ax)
    
    (mask_bands, mask_characters, mask_groups) = convert_selections(bands, characters, groups)

    # TODO: switch button for ignore_atoms_per_group
    ignore_atoms_per_group = True #default
    spins = [0]
    
    bandplotter.plot_bands(mask_bands, mask_characters, mask_groups, spins, 
         unfolding_weight_exponent, compare_characters, ax, ignore_atoms_per_group, marker_size)
    bandplotter.plot_dos(filepaths_dos[0], mask_groups, mask_characters,
           dos_groups_yesno, interstitial, all_characters,
               ax=ax_dos, fix_xlim=True)

FigureCanvasNbAgg()

interactive(children=(IntRangeSlider(value=(1, 303), continuous_update=False, description='Bands', max=303, mi…

# CURRENT layout

In [8]:
# Trying new layout with gridspec.
# Reference: https://plot.ly/matplotlib/subplots/
# Adjust GridSpec layout:
# https://matplotlib.org/users/gridspec.html


from matplotlib import gridspec

fig_scale = 0.65
fig_ratio = [12,6]
figsize=[fig_scale * el for el in fig_ratio]

fig = plt.figure(figsize=figsize)
# order: first add dos plot, then band plot.
# otherwise labels (.setup() below) will be set on dos instead band plot.
gs_dos = gridspec.GridSpec(1, 2)
ax_dos = fig.add_subplot(gs_dos[0,1])
gs = gridspec.GridSpec(1, 2)
ax = fig.add_subplot(gs[0,0], sharey=ax_dos)
gs.update(wspace=0, left=0.1, right=1.4)
gs_dos.update(left=0.6, right=0.9, wspace=0)
plt.setp(ax_dos.get_yticklabels(), visible=False)
# gs.tight_layout(fig, rect=[0, 0, 1, 0.97])


# fig, (ax, ax_dos) = plt.subplots(1, 2, sharey=True, 
#                                  figsize=[fig_scale * el for el in fig_ratio])



plt.suptitle(f"BandStructure of {filename}")


@interact(bands=select_bands, characters=select_characters, groups=select_groups, 
          unfolding_weight_exponent=select_exponent, marker_size=select_marker_size,
         compare_characters=select_compare_characters,
         dos_groups_yesno=True, interstitial=True, all_characters=True)
def update_plot(bands, characters, groups, 
                unfolding_weight_exponent, marker_size,
               compare_characters,
               dos_groups_yesno, interstitial, all_characters):
    ax.clear()
    ax_dos.clear()
#     bandplotter.setup(ax)
    
    
    (mask_bands, mask_characters, mask_groups) = convert_selections(bands, characters, groups)

    # TODO: switch button for ignore_atoms_per_group
    ignore_atoms_per_group = True #default
    spin = 0
    spins = [spin]

    bandplotter.setup(plt)
    bandplotter.plot_bands(mask_bands, mask_characters, mask_groups, spins, 
         unfolding_weight_exponent, compare_characters, ax, ignore_atoms_per_group, marker_size)
    bandplotter.plot_dos(filepaths_dos[spin], mask_groups, mask_characters,
           dos_groups_yesno, interstitial, all_characters,
               ax=ax_dos, fix_xlim=True)
#     bandplotter.plot_dos(filepaths_dos[spin], ax=ax_dos, only_total=False)

FigureCanvasNbAgg()

interactive(children=(IntRangeSlider(value=(1, 303), continuous_update=False, description='Bands', max=303, mi…

# Exploring DOS files

In [7]:




# find out: DOS file column numbers, vs. band file number of groups and characters
from collections import namedtuple

Info = namedtuple('Info', ['DOS_columns', 'band_groups', 'band_characters'])
Infos = {'mose' : None, 'co1' : None, 'co2' : None}

dosdata = {
    'mose' : ['..', 'data', 'input', 'MoSe2', "DOS.1"],
    'co1': ['..', 'data', 'input', 'Co', "DOS.1"],
    'co2': ['..', 'data', 'input', 'Co', "DOS.2"]
}
for k,v in dosdata.items():
    dosdata[k] = os.path.join(*v)
    dosdata[k] = np.genfromtxt(dosdata[k]).T
#     print(f"{k} columns x rows: {dosdata[k].shape}")

# # with pandas instead numpy:
# import pandas as pd
# df = pd.read_csv(test_dospath, sep='\s+')
# header = df.columns[1:] if (df.columns[0].startswith('#')) else df.columns
# df = pd.read_csv(test_dospath, sep='\s+', comment='#', header=None).T

banddata_paths = {
    'mose' : ['..', 'data', 'input', 'MoSe2', "banddos_2spin.hdf"],
    'co': ['..', 'data', 'input', 'Co', "banddos_Co.hdf"],
}
banddata = {}
for k,v in banddata_paths.items():
    banddata_paths[k] = os.path.join(*v)
    reader = Reader(banddata_paths[k])
    with reader as h5file:
        banddata[k] = reader.read(recipe=Recipes.Bands)
        banddata[k].move_datasets_to_memory()
#     print(f"banddata {k}: num_groups {banddata[k].num_groups}")
#     print(f"banddata {k}: num_chars {banddata[k].num_char}")
    


Infos['mose'] = Info(
    DOS_columns=dosdata['mose'].shape[0], 
    band_groups=banddata['mose'].num_groups, 
    band_characters=banddata['mose'].num_char)
Infos['co1'] = Info(
    DOS_columns=dosdata['co1'].shape[0], 
    band_groups=banddata['co'].num_groups, 
    band_characters=banddata['co'].num_char)
Infos['co2'] = Info(
    DOS_columns=dosdata['co2'].shape[0], 
    band_groups=banddata['co'].num_groups, 
    band_characters=banddata['co'].num_char)

# print(f"MoSe2: num_groups * 5 + 5 = {Infos['mose'].band_groups} * 5 + 5 = {Infos['mose'].band_groups * 5 + 5}")
# print(f"MoSe2: DOS_columns {Infos['mose'].DOS_columns}")
# print(f"Co   : num_groups * 5 + 5 = {Infos['co1'].band_groups} * 5 + 5 = {Infos['co1'].band_groups * 5 + 5}")
# print(f"Co   : DOS_columns {Infos['co1'].DOS_columns}")
# print(f"But MoSe2 and Co DOS files both do only have spd and not spdf set of characters")

# # check if group1 section2 equals section3
# d = dosdata['co1']
# np.array_equal(d[6], d[8]+d[10]+d[12]+d[14])
# # output: False


d = dosdata['co1']
num_groups = banddata['co'].num_groups
num_chars = banddata['co'].num_char
indices={
    'section1' : list(range(5)),
    'section2' : list(range(5, 5+num_groups)),
    'section3' : [[5+num_groups+(g) for g in range(c*num_groups,(c+1)*num_groups)] for c in range(num_chars)]
}
indices['section3']

[[7, 8], [9, 10], [11, 12], [13, 14]]